## Base models

In this notebook we will use some regression models to our data WITHOUT any tuning and data preprocessing (except missing value filling, encoding and scaling). Results which we will receive will be our base results which we will try to upgrade in the next notebooks

We will use 4 baseline models. DummyRegressor, Linear Regression (OLS), Ridge regression, and RandomForest regression. But it doesn't mean that we will not use some other models in future

In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib as mpl
import seaborn as sns
from IPython.core.display import HTML
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, root_mean_squared_error

## Content

In [83]:
data = pd.read_csv("data/train_without_nans.csv")
sub = pd.read_csv("data/test_without_nans.csv")

encoded_data = pd.read_csv("data/encoded_train.csv")
encoded_sub = pd.read_csv("data/encoded_test.csv")

scaled_data = pd.read_csv("data/scaled_train.csv")
scaled_sub = pd.read_csv("data/scaled_test.csv")

### Train/test split & scaling

In [84]:
X = encoded_data.drop(["SalePrice"], axis=1)
y = encoded_data["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

continuous_columns = data.select_dtypes(exclude="object").drop(["SalePrice", "Id", "MSSubClass"], axis=1).columns.values

print(continuous_columns)

scaler = StandardScaler()

X_train[continuous_columns] = scaler.fit_transform(X_train[continuous_columns])
X_test[continuous_columns] = scaler.transform(X_test[continuous_columns])

real_y_train = y_train.copy()
real_y_test = y_test.copy()

y_train = np.log(y_train)
y_test = np.log(y_test)

X_train

['LotFrontage' 'LotArea' 'OverallQual' 'OverallCond' 'YearBuilt'
 'YearRemodAdd' 'MasVnrArea' 'BsmtFinSF1' 'BsmtFinSF2' 'BsmtUnfSF'
 'TotalBsmtSF' '1stFlrSF' '2ndFlrSF' 'LowQualFinSF' 'GrLivArea'
 'BsmtFullBath' 'BsmtHalfBath' 'FullBath' 'HalfBath' 'BedroomAbvGr'
 'KitchenAbvGr' 'TotRmsAbvGrd' 'Fireplaces' 'GarageYrBlt' 'GarageCars'
 'GarageArea' 'WoodDeckSF' 'OpenPorchSF' 'EnclosedPorch' '3SsnPorch'
 'ScreenPorch' 'PoolArea' 'MiscVal' 'MoSold' 'YrSold']


,LotFrontage,LotArea,Street,Utilities,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_160,MSSubClass_180,MSSubClass_190
757,-1.635622,0.108491,1.0,0.0,-0.101069,-0.526850,0.206054,-0.364762,1.227466,-0.003617,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
598,0.642657,0.246038,1.0,0.0,-0.829142,0.384015,0.172740,-0.413590,1.945811,1.813417,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
215,0.414829,-0.052885,1.0,0.0,-0.829142,0.384015,-0.493554,0.514132,-0.220193,-0.171343,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
135,0.642657,-0.013773,1.0,0.0,0.627004,0.384015,-0.060463,-0.755382,1.008124,-0.945464,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
372,-0.211697,-0.338034,1.0,0.0,-0.101069,-0.526850,0.405942,-0.071798,-0.571140,0.394195,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,0.585700,-0.039915,1.0,0.0,-0.101069,-0.526850,0.072796,-0.560072,0.382998,-0.945464,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
905,0.642657,-0.062035,1.0,0.0,-0.829142,-0.526850,-0.593498,-1.536621,0.032051,-0.184245,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1096,0.073087,-0.367494,1.0,0.0,-0.101069,1.294880,-1.926086,1.002406,-0.571140,-0.945464,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235,-1.037574,-0.890535,1.0,0.0,-0.101069,-2.348579,-0.027148,-0.706555,2.740927,-0.175644,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## DummyRegressor

In [85]:
from sklearn.dummy import DummyRegressor

dummy_reg = DummyRegressor().fit(X_train, y_train)

train_pred = dummy_reg.predict(X_train)
test_pred = dummy_reg.predict(X_test)

print("TRAIN LOG RMSE:", root_mean_squared_error(y_train, train_pred))
print("TEST LOG RMSE:", root_mean_squared_error(y_test, test_pred))
print("."*40)
print("TRAIN RMSE", root_mean_squared_error(real_y_train, np.exp(train_pred)))
print("TEST RMSE", root_mean_squared_error(real_y_test, np.exp(test_pred)))

TRAIN LOG RMSE: 0.39752651637288144
TEST LOG RMSE: 0.4074188586002447
........................................
TRAIN RMSE 81724.9360837574
TEST RMSE 75334.24405265198


This will be our "dummy" baseline

## OLS

In [86]:
from sklearn.linear_model import LinearRegression

lr_reg = LinearRegression().fit(X_train, y_train)

train_pred = lr_reg.predict(X_train)
test_pred = lr_reg.predict(X_test)

print("TRAIN LOG RMSE:", root_mean_squared_error(y_train, train_pred))
print("TEST LOG RMSE:", root_mean_squared_error(y_test, test_pred))

TRAIN LOG RMSE: 0.08901084902093606
TEST LOG RMSE: 1486858776.871644


What we see? Abnormally large test error. Even when we used logarithmic scaling for the target which means our model must give us not large values. Let's check minimal and maximal predicted values for train and test

In [87]:
print("MIN TRAIN PREDICTION", np.min(train_pred))
print("MIN TEST PREDICTION", np.min(test_pred))
print("MAX TRAIN PREDICTION", np.max(train_pred))
print("MAX TEST PREDICTION", np.max(test_pred))

MIN TRAIN PREDICTION 10.640533447265625
MIN TEST PREDICTION -22993138933.899933
MAX TRAIN PREDICTION 13.55511474609375
MAX TEST PREDICTION 7606283384.342773


Probably this is the problem of a strong overfitting. Let's check model's largest coefficients

In [88]:
sorted_coef_idx = np.argsort(-np.abs(lr_reg.coef_))
coef = lr_reg.coef_[sorted_coef_idx]
for i in range(0, 10):
    print("{:.<025} {:< 010.4e}".format(X_train.columns[sorted_coef_idx[i]], coef[i]))

GrLivArea................  1.4165e+11
BsmtFinSF1...............  1.3885e+11
TotalBsmtSF.............. -1.3487e+11
BsmtUnfSF................  1.3446e+11
2ndFlrSF................. -1.1659e+11
1stFlrSF................. -1.0700e+11
GarageFinish_Unf......... -9.9042e+10
GarageFinish_Fin......... -9.9042e+10
GarageFinish_RFn......... -9.9042e+10
BldgType_2fmCon..........  7.4781e+10


We have very large coefficients near some features. This can be the problem of high correlation between some features.
BUT we can handle this problem not only with deleting correlated features. Also we can use L1 and L2 regularization

## Ridge Regression

In [89]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge().fit(X_train, y_train)

train_pred = ridge_reg.predict(X_train)
test_pred = ridge_reg.predict(X_test)

print("TRAIN LOG RMSE:", root_mean_squared_error(y_train, train_pred))
print("TEST LOG RMSE:", root_mean_squared_error(y_test, test_pred))
print("." * 35)
print("TRAIN RMSE", root_mean_squared_error(real_y_train, np.exp(train_pred)))
print("TEST RMSE", root_mean_squared_error(real_y_test, np.exp(test_pred)))

TRAIN LOG RMSE: 0.09618528732536605
TEST LOG RMSE: 0.12981291831482283
...................................
TRAIN RMSE 19857.26655927968
TEST RMSE 21367.482670061567


## RandomForest regression

In [90]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor().fit(X_train, y_train)

train_pred = rf_reg.predict(X_train)
test_pred = rf_reg.predict(X_test)

print("TRAIN LOG RMSE:", root_mean_squared_error(y_train, train_pred))
print("TEST LOG RMSE:", root_mean_squared_error(y_test, test_pred))
print("." * 35)
print("TRAIN RMSE", root_mean_squared_error(real_y_train, np.exp(train_pred)))
print("TEST RMSE", root_mean_squared_error(real_y_test, np.exp(test_pred)))

TRAIN LOG RMSE: 0.05533902955344705
TEST LOG RMSE: 0.13739435320337184
...................................
TRAIN RMSE 11790.3454945595
TEST RMSE 22874.72000592039


## Submission to kaggle

Before submission we will use the whole train dataset for training. Because of the fact that ols gives us overflow we will submit onlu ridge and random_forest

In [108]:
scaled_X = scaled_data.drop(["SalePrice"], axis=1)
scaled_y = scaled_data["SalePrice"]

ridge_reg = ridge_reg.fit(scaled_X, scaled_y)
ridge_sub = pd.Series(np.exp(ridge_reg.predict(scaled_sub)))
ridge_sub_df = pd.DataFrame()
ridge_sub_df["Id"] = sub["Id"]
ridge_sub_df["SalePrice"] = ridge_sub

ridge_sub_df.to_csv("submissions/baseline/linear_ridge.csv", index=False)


rf_reg = rf_reg.fit(scaled_X, scaled_y)
rf_sub = pd.Series(np.exp(rf_reg.predict(scaled_sub)))
rf_sub_df = pd.DataFrame()
rf_sub_df["Id"] = sub["Id"]
rf_sub_df["SalePrice"] = rf_sub

rf_sub_df.to_csv("submissions/baseline/random_forest.csv", index=False)